In [110]:
from b2heavy.ThreePointFunctions.corr3pts import BINSIZE

from b2heavy.TwoPointFunctions.utils     import Tmax
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO, Correlator
from b2heavy.TwoPointFunctions.fitter    import StagFitter, p_value, standard_p

from b2heavy.FnalHISQMetadata import params

import lsqfit
import copy
import h5py

import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt
import pandas            as pd
import corrfitter        as cf

from tqdm  import tqdm
from scipy import linalg as la

In [69]:
def fexp(Nt):
    return lambda t,E,Z: Z * ( np.exp(-E*t) + np.exp(-E*(Nt-t)) ) 

In [70]:
ensemble = 'Coarse-1'
mom      = '300'
mes      = 'Dst'
data_dir = '/Users/pietro/code/data_analysis/BtoD/Alex/'

mdata = params(ensemble)

cov_specs = dict(scale=True, shrink=True, cutsvd=1e-12)
jkb       = BINSIZE[ensemble]

In [71]:
io   = CorrelatorIO(ensemble,'Dst',mom,PathToDataDir=data_dir)
stag = StagFitter(
    io       = io,
    jkBin    = BINSIZE[ensemble],
    smearing = ['1S-1S','d-d','d-1S']
)

In [72]:
teff    = (10,20)
tmin    = int(0.65//mdata['aSpc'].mean)
Nstates = 3

In [73]:
effm,effa = stag.meff(teff,**cov_specs)

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


# define function

### version 1

In [74]:
def corr2(mes,Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        erg    = p[f'dE.{mes}'  ]**2
        ergo   = p[f'dE.{mes}.o']**2
        erg[0] = p[f'dE.{mes}'][0]

        c2 = 0.
        for n in range(Nstates):
            Z0 = p[f'Z.{mes}.{sm1}.{pol}'  ][n]**2 * p[f'Z.{mes}.{sm2}.{pol}'  ][n]**2
            Z1 = p[f'Z.{mes}.{sm1}.{pol}.o'][n]**2 * p[f'Z.{mes}.{sm2}.{pol}.o'][n]**2

            if n>0 and mix:
                Z0 = p[f'Z.{mes}.{sm}.{pol}'  ][n-1]**2
                Z1 = p[f'Z.{mes}.{sm}.{pol}.o'][n-1]**2

            Ephy = sum(erg[ :n+1])
            Eosc = sum(ergo[:n+1])
            
            c2 += fexp(Nt)(t,Ephy,Z0) + fexp(Nt)(t,Eosc,Z1) * (-1)**((t+1))

        return c2
    
    return aux

In [75]:
pr = {}
for smr,pol in effa:
    s1,s2 = smr.split('-')
    if s1==s2:
        aux = np.sqrt(effa[smr,pol])
        z0   = f'{aux.mean:.6f}(1.0)'
        zexc = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(Nstates)]

        pr[f'Z.{mes}.{s1}.{pol}']    = zexc
        pr[f'Z.{mes}.{s1}.{pol}'][0] = z0 
        pr[f'Z.{mes}.{s1}.{pol}.o']  = zexc

    else:
        pr[f'Z.{mes}.{smr}.{pol}']   = zexc[1:]
        pr[f'Z.{mes}.{smr}.{pol}.o'] = zexc[1:]

e = '0.7(5)'
pr[f'dE.{mes}']   = [e for n in range(Nstates)]; pr[f'dE.{mes}'  ][0] = f'{effm.mean:.2f}(0.7)'
pr[f'dE.{mes}.o'] = [e for n in range(Nstates)]; pr[f'dE.{mes}.o'][0] = f'{effm.mean:.2f}(0.7)'

prior = gv.BufferDict()
for k,p in pr.items():
    prior[k] = gv.gvar(p)
    print(k,prior[k])

Z.Dst.1S.Bot [0.9(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Bot.o [0.9(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par [1.1(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par.o [1.1(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Bot [1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Bot.o [1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Par [1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Par.o [1.0(1.0) 1.0(1.0)]
Z.Dst.d.Bot [0.1(1.0) 0.01(50) 0.01(50)]
Z.Dst.d.Bot.o [0.1(1.0) 0.01(50) 0.01(50)]
Z.Dst.d.Par [0.2(1.0) 0.01(50) 0.01(50)]
Z.Dst.d.Par.o [0.2(1.0) 0.01(50) 0.01(50)]
dE.Dst [1.20(70) 0.70(50) 0.70(50)]
dE.Dst.o [1.20(70) 0.70(50) 0.70(50)]


### version 0

In [98]:
def corr2(mes,Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        erg    = np.exp(p[f'dE.{mes}'  ])
        ergo   = np.exp(p[f'dE.{mes}.o'])
        erg [0] = p[f'dE.{mes}'  ][0]
        ergo[0] = np.exp(p[f'dE.{mes}.o'][0]) + erg[0]

        c2 = 0.
        for n in range(Nstates):
            Z0 = np.exp(p[f'Z.{mes}.{sm1}.{pol}'  ][n]) * np.exp(p[f'Z.{mes}.{sm2}.{pol}'  ][n])
            Z1 = np.exp(p[f'Z.{mes}.{sm1}.{pol}.o'][n]) * np.exp(p[f'Z.{mes}.{sm2}.{pol}.o'][n])

            if n>0 and mix:
                Z0 = p[f'Z.{mes}.{sm}.{pol}'  ][n-1]**2
                Z1 = p[f'Z.{mes}.{sm}.{pol}.o'][n-1]**2

            Ephy = sum(erg[ :n+1])
            Eosc = sum(ergo[:n+1])
            
            c2 += fexp(Nt)(t,Ephy,Z0) + fexp(Nt)(t,Eosc,Z1) * (-1)**((t+1))

        return c2
    
    return aux

In [99]:
pr = {}
for smr,pol in effa:
    s1,s2 = smr.split('-')
    if s1==s2:
        aux = np.log(np.sqrt(effa[smr,pol]))
        z0   = f'{aux.mean:.6f}(1.0)'
        zexc = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(Nstates)]

        pr[f'Z.{mes}.{s1}.{pol}']    = zexc
        pr[f'Z.{mes}.{s1}.{pol}'][0] = z0 
        pr[f'Z.{mes}.{s1}.{pol}.o']  = zexc

    else:
        pr[f'Z.{mes}.{smr}.{pol}']   = zexc[1:]
        pr[f'Z.{mes}.{smr}.{pol}.o'] = zexc[1:]

e = '-0.7(1.0)'
pr[f'dE.{mes}']   = [e for n in range(Nstates)]; pr[f'dE.{mes}'  ][0] = f'{np.log(effm.mean):.2f}(0.5)'
pr[f'dE.{mes}.o'] = [e for n in range(Nstates)]; #pr[f'dE.{mes}.o'][0] = f'{np.log(effm.mean):.2f}(1.0)'

prior = gv.BufferDict()
for k,p in pr.items():
    prior[k] = gv.gvar(p)
    print(k,prior[k])

Z.Dst.1S.Bot [-0.1(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Bot.o [-0.1(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par [0.05(1.00) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par.o [0.05(1.00) 1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Bot [1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Bot.o [1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Par [1.0(1.0) 1.0(1.0)]
Z.Dst.d-1S.Par.o [1.0(1.0) 1.0(1.0)]
Z.Dst.d.Bot [-2.0(1.0) 0.01(50) 0.01(50)]
Z.Dst.d.Bot.o [-2.0(1.0) 0.01(50) 0.01(50)]
Z.Dst.d.Par [-1.9(1.0) 0.01(50) 0.01(50)]
Z.Dst.d.Par.o [-1.9(1.0) 0.01(50) 0.01(50)]
dE.Dst [0.18(50) -0.7(1.0) -0.7(1.0)]
dE.Dst.o [-0.7(1.0) -0.7(1.0) -0.7(1.0)]


## format data

In [100]:
xdata, ydata, yfull = stag.format(alljk=True,**cov_specs)

In [101]:
xfit, yfit, yjk = {},{},{}
for smr,pol in stag.keys:
    tmax = Tmax(ydata[smr,pol])

    xfit[smr,pol] = xdata[tmin:tmax]
    yfit[smr,pol] = ydata[smr,pol][tmin:tmax]
    yjk [smr,pol] = yfull[smr,pol][:,tmin:tmax]

yflat   = np.concatenate([yfit[k] for k in stag.keys])
yflatjk = np.hstack([yjk[k] for k in stag.keys])

In [102]:
def fitfcn(xd,p):
    tmp = []
    for sm,pl in stag.keys:
        fcn = corr2('Dst',Nstates,stag.Nt,sm,pl)
        tmp.append(fcn(xfit[sm,pl],p))
    return np.concatenate(tmp)

## do fit

In [103]:
fit = lsqfit.nonlinear_fit(
    data  = (xfit,yflat),
    fcn   = fitfcn,
    prior = prior
)

In [104]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 1.4 [78]    Q = 0.02    logGBF = 1257.6

Parameters:
    Z.Dst.1S.Bot 0    -0.32 (12)     [  -0.1 (1.0) ]  
                 1    0.082 (69)     [   1.0 (1.0) ]  
                 2     0.69 (86)     [   1.0 (1.0) ]  
  Z.Dst.1S.Bot.o 0    -0.43 (14)     [  -0.1 (1.0) ]  
                 1     0.14 (15)     [   1.0 (1.0) ]  
                 2     1.60 (52)     [   1.0 (1.0) ]  
    Z.Dst.1S.Par 0    -0.15 (12)     [ 0.05 (1.00) ]  
                 1    0.234 (70)     [   1.0 (1.0) ]  
                 2     1.14 (50)     [   1.0 (1.0) ]  
  Z.Dst.1S.Par.o 0    -0.76 (14)     [ 0.05 (1.00) ]  
                 1    -0.06 (14)     [   1.0 (1.0) ]  *
                 2     1.07 (90)     [   1.0 (1.0) ]  
  Z.Dst.d-1S.Bot 0    0.405 (29)     [   1.0 (1.0) ]  
                 1     0.76 (53)     [   1.0 (1.0) ]  
Z.Dst.d-1S.Bot.o 0    0.463 (47)     [   1.0 (1.0) ]  
                 1     1.13 (77)     [   1.0 (1.0) ]  
  Z.Dst.d-1S.Par 0    0.478 (

In [109]:
aux,idx = [],[]
for k in fit.p:
    if k.endswith('o'):
        continue


    tmp = {}
    for n in range(len(fit.p[k])):
        ise0 = (n==0 and k.startswith('dE') and not k.endswith('o'))
        mix = '-' in k

        fcn = lambda x: (x if ise0 else np.exp(x))

        tmp[f'{n+1 if mix else n}']   = fcn(fit.p[k][n])
        tmp[f'{n+1 if mix else n}.o'] = fcn(fit.p[f'{k}.o'][n])


    aux.append(tmp)
    idx.append(k)

pd.DataFrame(aux,index=idx).transpose()

,Z.Dst.1S.Bot,Z.Dst.1S.Par,Z.Dst.d-1S.Bot,Z.Dst.d-1S.Par,Z.Dst.d.Bot,Z.Dst.d.Par,dE.Dst
0,0.728(88),0.86(10),NaN,NaN,0.108(14),0.123(16),1.180(15)
0.o,0.651(91),0.465(66),NaN,NaN,0.0630(92),0.0100(42),-2.98(53)
1,1.086(75),1.264(88),1.499(43),1.613(53),0.180(16),0.204(17),0.245(43)
1.o,1.15(18),0.94(13),1.589(75),1.543(50),0.199(18),0.195(19),0.288(35)
2,2.0(1.7),3.1(1.6),2.1(1.1),1.8(1.3),1.37(42),1.38(43),0.93(12)
2.o,5.0(2.6),2.9(2.6),3.1(2.4),2.2(1.9),1.03(37),0.87(33),1.00(17)


## jkfit

In [94]:
njk = yflatjk.shape[0]
cov = gv.evalcov(yflat)

aux = []
for ijk in tqdm(range(njk)):
    yf = yflatjk[ijk,:]

    jfit = lsqfit.nonlinear_fit(
        data  = (xfit,yf,cov),
        fcn   = fitfcn,
        prior = gv.gvar(gv.mean(fit.p),gv.sdev(prior))
    )

    aux.append(
        {k: jfit.pmean[k][0] for k in jfit.p}
    )

100%|██████████| 90/90 [00:08<00:00, 10.02it/s]


In [97]:
df = pd.DataFrame(aux)
df

,Z.Dst.1S.Bot,Z.Dst.1S.Bot.o,Z.Dst.1S.Par,Z.Dst.1S.Par.o,Z.Dst.d-1S.Bot,Z.Dst.d-1S.Bot.o,Z.Dst.d-1S.Par,Z.Dst.d-1S.Par.o,Z.Dst.d.Bot,Z.Dst.d.Bot.o,Z.Dst.d.Par,Z.Dst.d.Par.o,dE.Dst,dE.Dst.o
0,-0.202716,-0.710784,-0.033046,-1.056935,0.371306,0.473919,0.443828,0.403230,-2.098834,-3.079012,-1.969012,-5.603870,1.197369,1.186206
1,-0.222849,-0.740425,-0.053809,-1.095139,0.374607,0.478146,0.444957,0.403719,-2.122335,-3.106896,-1.990967,-5.610024,1.194803,1.180721
2,-0.212487,-0.733727,-0.041206,-1.083866,0.373229,0.473046,0.442302,0.401045,-2.109865,-3.099633,-1.978607,-5.647304,1.196480,1.181775
3,-0.209004,-0.701976,-0.040463,-1.041840,0.373550,0.475178,0.445980,0.404660,-2.104839,-3.072718,-1.973418,-5.535239,1.196652,1.187449
4,-0.206833,-0.698828,-0.036908,-1.048490,0.371397,0.472624,0.442552,0.404047,-2.102601,-3.071795,-1.972761,-5.627308,1.197226,1.187444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,-0.201043,-0.723252,-0.031433,-1.074802,0.373536,0.474241,0.446595,0.402090,-2.097035,-3.088122,-1.966883,-5.563600,1.197626,1.183643
86,-0.228408,-0.708786,-0.056622,-1.052640,0.380895,0.473795,0.450077,0.403144,-2.126066,-3.072797,-1.994825,-5.547157,1.194074,1.186118
87,-0.198448,-0.699910,-0.028898,-1.041109,0.372768,0.467398,0.443482,0.399180,-2.097215,-3.065733,-1.965866,-5.586631,1.197924,1.188167
88,-0.199189,-0.700268,-0.030112,-1.046818,0.367408,0.470151,0.439142,0.400412,-2.094961,-3.066569,-1.963895,-5.555414,1.198214,1.188101


In [95]:
df = pd.DataFrame(aux)

fitp = {}
fjk = gv.gvar(
    df.mean().values,
    df.cov().values * (njk-1)
)
for i,col in enumerate(df.columns):
    ise0 = n==0 and k.startswith('dE')

    fcn = lambda x: x if ise0 else np.exp(x)
    fitp[col] = fcn(fjk[i])


fitp

{'Z.Dst.1S.Bot': 0.809(66),
 'Z.Dst.1S.Bot.o': 0.494(69),
 'Z.Dst.1S.Par': 0.959(75),
 'Z.Dst.1S.Par.o': 0.349(49),
 'Z.Dst.d-1S.Bot': 1.456(54),
 'Z.Dst.d-1S.Bot.o': 1.603(45),
 'Z.Dst.d-1S.Par': 1.565(63),
 'Z.Dst.d-1S.Par.o': 1.494(41),
 'Z.Dst.d.Bot': 0.121(10),
 'Z.Dst.d.Bot.o': 0.0463(62),
 'Z.Dst.d.Par': 0.138(12),
 'Z.Dst.d.Par.o': 0.0038(11),
 'dE.Dst': 3.307(38),
 'dE.Dst.o': 3.278(80)}